In [1]:
## Importing modules
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
from gseapy.parser import Biomart
import os
import numpy as np
import seaborn as sns
from gseapy.plot import gseaplot


## Setting base directory

In [2]:
Base_dir='/data/nandas/Combined_coexp/Pathway_enrichment/NewSets_090420/OverallPathwayEnrichmentmean033122'
os.chdir(Base_dir)

In [3]:
# ! mkdir /data/nandas/Combined_coexp/Pathway_enrichment/NewSets_090420/OverallPathwayEnrichmentmean033122

## Reading required files

In [4]:
pathway_filename = '/data/nandas/Combined_coexp/Pathway_enrichment/NewSets_090420/Genesets_NAME_090320_LEVEL_4.gmt';
metabolic_corr_df=pd.read_csv("/data/nandas/Combined_coexp/Sleipnir/Final_data_080620/UMN/MetabolicCorrMatrix_083120.csv",index_col=0,header='infer')
Pathway_df=pd.read_csv(pathway_filename,index_col=0,sep='\t')



In [61]:
# ## PreRank Gene set enrichment analyses for custom pathway annotations

# In[60]:
def wb_to_gene(matrix):
    mapper_df=pd.read_csv("/data/nandas/WormBase_282/MasterProteinCodingGenesAnnotation_WS282.csv", 
                          header='infer',index_col=1)
    mapper_df=mapper_df.loc[mapper_df.index.dropna()]
    wb_to_gene = {};
    for wb in mapper_df.index:
        wb_to_gene[wb] = str(mapper_df.loc[wb]['GeneName']);
    matrix=matrix.rename(index=wb_to_gene,columns=wb_to_gene)
    return matrix

def gene_to_wb(matrix):
    mapper_df=pd.read_csv("/data/nandas/WormBase_282/MasterProteinCodingGenesAnnotation_WS282.csv",
                          header='infer',index_col=2)
    mapper_df=mapper_df.loc[mapper_df.index.dropna()]
    gene_to_wb = {};
    for gene in mapper_df.index:
        gene_to_wb[gene] = str(mapper_df.loc[gene]['WormBaseID']);
    matrix=matrix.rename(index=gene_to_wb,columns=gene_to_wb)
    return matrix

def SeqToWB(output_df):
    mapper_df=pd.read_csv("/data/nandas/WormBase_282/MasterProteinCodingGenesAnnotation_WS282.csv",
                          header='infer',index_col=3)
    mapper_df=mapper_df.loc[mapper_df.index.dropna()]
    Seq_to_Wb = {};
    mapper_df=mapper_df[mapper_df.index!=np.nan]
    for seq in mapper_df.index:
        Seq_to_Wb[seq] = str(mapper_df.loc[seq]['WormBaseID']);
    matrix=matrix.rename(index=Seq_to_Wb,columns=Seq_to_Wb)
    return matrix

def SeqToGene(matrix):
    mapper_df=pd.read_csv("/data/nandas/WormBase_282/MasterProteinCodingGenesAnnotation_WS282.csv", 
                          header='infer',index_col=3)
    mapper_df=mapper_df.loc[mapper_df.index.dropna()]
    Seq_to_Gene = {};
    mapper_df=mapper_df[mapper_df.index!=np.nan]
    for seq in mapper_df.index:
        Seq_to_Gene[seq] = str(mapper_df.loc[seq]['GeneName']);
    matrix=matrix.rename(index=Seq_to_Gene,columns=Seq_to_Gene)
    return matrix

def GeneToSeq(matrix):
    mapper_df=pd.read_csv("/data/nandas/WormBase_282/MasterProteinCodingGenesAnnotation_WS282.csv", 
                          header='infer',
                          index_col=2)
    mapper_df=mapper_df.loc[mapper_df.index.dropna()]
    Gene_to_Seq = {};
    mapper_df=mapper_df[mapper_df.index!=np.nan]
    for gene in mapper_df.index:
        Gene_to_Seq[gene] = str(mapper_df.loc[gene]['SequenceID']);
    matrix=matrix.rename(index=Gene_to_Seq,columns=Gene_to_Seq)
    return matrix

def PreRank(genes, outdir,gene_sets):
#     print("Genes: {}".format(genes));
    print("Length of genes:{}".format(len(genes)))
    genes=pd.DataFrame(genes)
    genes.set_index([0],inplace=True)
    genes=SeqToGene(genes)
    genes=list(genes.index)
    intersection_list = list(set(metabolic_corr_df.index).intersection(set(genes)))
#     print("intersection_list:{}".format(intersection_list))
    missing_genes=list(set(genes).difference(set(intersection_list)))
#     print("IntersectionList: {}".format(intersection_list));
#     print("Length of intersection list:{}".format(len(intersection_list)))
#     print('Missing genes:{}\n{}'.format(len(missing_genes),missing_genes))
    if(len(missing_genes) == len(genes)):
        return;
    Combined=metabolic_corr_df[intersection_list];
    Mean=Combined.mean(axis=1)
#    print("Mean before scaling:{}".format(Mean))
#     Mean=(Mean*2)-1
#     print("Mean after scaling to lie between -1 and +1:{}".format(Mean))
    Mean.dropna(inplace=True)
    rnk=Mean.sort_values(ascending=False)
    plt.rcParams["font.family"] = "Arial"
#     print("Rank: {}".format(rnk))    
    pre_res = gp.prerank(rnk=rnk, gene_sets=gene_sets, processes=4,min_size=2, outdir=outdir, format='svg', 
                         weighted_score_type=1,verbose=True)
    plt.close()
    return pre_res

def _is_regulated_pathway_(pre_res, pathway):
#     print('Hello There: {}'.format(pre_res));
#     print('Shivani Here: {}'.format(pre_res.res2d))
    if(pathway not in pre_res.res2d.index):
        return "NaN"
    else:
        pathway_pre_res = pre_res.res2d.loc[pathway];
#     is_regulated_pathway = pathway_pre_res.es >= 0.70 and pathway_pre_res.fdr <= 0.05
        is_regulated_pathway =  (pathway_pre_res.fdr <= 0.05) and (pathway_pre_res.nes>0) and (pathway_pre_res.nes!=np.inf) and (pathway_pre_res.es>0)
    return is_regulated_pathway;

def PlotEnrichment(pre_res,pathway, outdir):
    Sorted_values=pre_res.res2d.sort_values(ascending=False,by=['nes'])[0:40]
    fig = plt.figure(figsize=(8,15))
    df = pd.DataFrame({'Enrichment Score': Sorted_values.es,
                   'p-value': Sorted_values.pval,'FDR':Sorted_values.fdr}, index=Sorted_values.index)
    ax = df.plot.barh(rot=0)
    plt.legend(loc='best', bbox_to_anchor=(1, 1))
    plt.rcParams["font.family"] = "Arial"
    plt.savefig("{}/{}_plot.svg".format(outdir, pathway))
    plt.show()
    plt.close()
    
def PlotGSEA(pre_res, pathway, outdir):
    terms = pre_res.res2d.sort_values(by=['es'],ascending=False).index
    print(terms[17])
    print(terms)
    fig=gseaplot(rank_metric=pre_res.ranking,term=terms[17], **pre_res.results[pathway],ofname='{}/{}_gsea.svg'.format(outdir,terms[16]))
    plt.close()
    

In [6]:
metabolic_corr_df=SeqToGene(metabolic_corr_df)

In [7]:
metabolic_corr_df=metabolic_corr_df[~metabolic_corr_df.index.duplicated(keep='first')]

In [8]:
# metabolic_corr_df=SeqToGene(metabolic_corr_df)

In [9]:
metabolic_corr_df=wb_to_gene(metabolic_corr_df)

In [10]:
# metabolic_corr_df=(metabolic_corr_df+1)/2

In [11]:
metabolic_corr_df.min().min()

0.374581

In [12]:
# ### Setting default coregulated state of pathway
Pathway_df['IsRegulated'] = False
Pathway_df

,Dummy,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,IsRegulated
Categories_Pathways,,,,,,,,,,,,,,,,,,,,,
ALA,NaN,B0205.6,C32F10.8,C44B7.7,F13H8.9,K10D2.7,T09B4.8,T27A3.6,agxt-1,kmo-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
AMINO_SUGAR_AND_NUCLEOTIDE_SUGAR_METABOLISM,NaN,C01F1.3,C08B6.4,C36A4.4,C50D2.7,D1005.2,F21D5.1,F59B2.3,K08E3.5,R05F9.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
AMINOACYL_TRNA_BIOSYNTHESIS,NaN,C39B5.6,Y105E8A.20,Y41D4A.6,Y66D12A.7,aars-1,aars-2,cars-1,dars-1,dars-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
ARG,NaN,C06A6.4,C10C5.3,C10C5.4,C10C5.5,C44E12.1,D2023.4,F32B5.1,F53F10.2,F55G1.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
ASCAROSIDE_BIOSYNTHESIS,NaN,acox-1.1,acox-1.2,acox-1.3,acox-1.4,acox-3,art-1,daf-22,dhs-28,elo-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VACUOLAR_ATP_ASE,NaN,spe-5,unc-32,vha-1,vha-10,vha-11,vha-12,vha-13,vha-14,vha-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
VAL_DEGRADATION,NaN,B0250.5,B0272.3,F54C8.1,T09B4.8,Y43F4A.4,Y44A6D.5,acdh-1,acdh-10,acdh-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
VITAMIN_B12_ENZYME,NaN,metr-1,mmcm-1,mtrr-1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [13]:
Pathway_df[Pathway_df.index.str.startswith("P")]

,Dummy,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,IsRegulated
Categories_Pathways,,,,,,,,,,,,,,,,,,,,,
PANTOTHENATE_AND_COA_BIOSYNTHESIS,NaN,F25H9.6,T05G5.5,Y65B4A.8,Y71H2AM.6,pnk-1,pnk-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS,NaN,C03A7.13,D1005.2,F10D2.8,F54C1.1,H23N18.4,K08E3.5,R04B5.5,R04B5.6,R08D7.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PENTOSE_PHOSPHATE_PATHWAY,NaN,F07A11.5,F08F8.7,F09E5.3,F26D11.1,R05F9.6,R151.2,T25B9.9,T25C8.1,Y43F4B.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PEROXISOMAL_FATTY_ACID_DEGRADATION,NaN,F53C11.3,F58A6.1,acox-1.1,acox-1.2,acox-1.3,acox-1.4,acox-1.5,acox-1.6,acox-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PHE,NaN,C31H2.4,bas-1,got-1.2,hdl-1,hpd-1,mif-1,nkat-1,prdx-6,tatn-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PORPHYRIN_METABOLISM,NaN,C03A7.13,F10D2.8,F54C1.1,H23N18.4,cchl-1,cox-10,cox-15,ugt-1,ugt-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PRO,NaN,B0513.5,C14E2.4,F55G1.9,M153.1,Y43F8B.19,alh-6,dpy-18,got-1.2,got-2.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PROPIONATE_BREAKDOWN_CANONICAL,NaN,mce-1,mmcm-1,pcca-1,pccb-1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
PROPIONATE_OTHER,NaN,B0303.3,B0395.3,Y43F4A.4,acaa-2,acs-19,bckd-1A,bckd-1B,dbt-1,dld-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [14]:
np.fill_diagonal(metabolic_corr_df.values,np.nan)

In [15]:
metabolic_corr_df=(metabolic_corr_df*2)-1

In [16]:
metabolic_corr_df.min().min()

-0.250838

In [17]:
metabolic_corr_df

,aap-1,ace-1,ace-2,ace-3,ace-4,aco-1,aco-2,acy-1,acy-2,acy-4,...,pigb-1,coa-1,F11C1.10,C05E4.15,C27A2.12,B0273.115,T25G12.13,F21A3.11,F36D3.16,dib-1
aap-1,NaN,0.039624,0.028208,0.034646,0.054360,-0.058982,0.072140,0.000348,0.049770,0.166650,...,0.325854,0.175490,0.032120,0.129924,-0.137186,0.064312,0.024504,0.005928,-0.047940,0.372246
ace-1,0.039624,NaN,0.178570,0.363616,0.204646,0.146422,0.202622,0.367574,0.294600,0.413470,...,0.089366,0.220314,0.242896,0.132284,0.205496,0.250636,0.142816,0.239896,0.032778,-0.046882
ace-2,0.028208,0.178570,NaN,0.184398,0.265924,0.125590,0.178284,0.155964,0.216902,0.136712,...,0.072898,0.189372,0.224584,0.234674,0.122876,0.244354,0.209356,0.106260,0.157978,0.004454
ace-3,0.034646,0.363616,0.184398,NaN,0.257020,0.151370,0.149156,0.412940,0.349712,0.240224,...,0.147090,0.275416,0.215496,0.141984,0.125650,0.269038,0.138384,0.194350,-0.042016,-0.025006
ace-4,0.054360,0.204646,0.265924,0.257020,NaN,0.068342,0.136878,0.154980,0.309200,0.094066,...,0.124790,0.118362,0.157554,0.208780,0.109284,0.223892,0.199628,0.081786,0.081988,0.006846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0273.115,0.064312,0.250636,0.244354,0.269038,0.223892,0.086454,0.023908,0.204074,0.280660,0.228540,...,0.152632,0.209842,0.245312,0.253874,0.239060,NaN,0.135092,0.256014,0.196022,0.090404
T25G12.13,0.024504,0.142816,0.209356,0.138384,0.199628,0.208606,0.123494,0.103148,0.180152,0.101980,...,0.059652,0.130860,0.200526,0.140242,0.174342,0.135092,NaN,0.123488,0.126406,0.112212
F21A3.11,0.005928,0.239896,0.106260,0.194350,0.081786,0.103512,0.122056,0.207046,0.210158,0.246998,...,0.122386,0.174728,0.164026,0.126940,0.206626,0.256014,0.123488,NaN,0.079212,0.027692
F36D3.16,-0.047940,0.032778,0.157978,-0.042016,0.081988,0.046438,0.120558,0.044588,0.088174,0.051194,...,0.048490,0.120690,0.154664,0.095156,0.167344,0.196022,0.126406,0.079212,NaN,-0.027806


In [26]:
# Pathway_df_withoutIsRegulated = Pathway_df.drop(['IsRegulated'], axis=1);
# New_df = pd.DataFrame([])
# for pathway in Pathway_df.index:
# #     pathway = 'PANTOTHENATE_AND_COA_BIOSYNTHESIS';
#     print(pathway)
# #     pathway = 'OTHER';
#     genes = list(Pathway_df_withoutIsRegulated.loc[pathway].dropna());
#     pre_res = PreRank(genes, pathway,gene_sets=pathway_filename);
#     if(pre_res is None):
#         continue; 
#     Pathway_df.at[pathway, 'IsRegulated'] = _is_regulated_pathway_(pre_res, pathway);
#     print("{} is regulated:{}".format(pathway,_is_regulated_pathway_(pre_res, pathway)))
#     PlotEnrichment(pre_res, pathway, outdir=pathway)
#     if(pathway in pre_res.res2d.index):
#         PlotGSEA(pre_res, pathway,pathway)
#         gsea_result_df=pre_res.res2d.loc[pathway];
        
#         New_df=New_df.append(gsea_result_df)
# #     break;
# # Pathway_df.to_csv("Pathway_Regulation_status_112821.csv")
# New_df.to_csv("Final_pathway_gsea_033122.csv")

ALA
Length of genes:12


KeyboardInterrupt: 

In [ ]:
Pathway_df_withoutIsRegulated = Pathway_df.drop(['IsRegulated'], axis=1);
New_df = pd.DataFrame([])
for pathway in Pathway_df.index:
    pathway = 'PROPIONATE_SHUNT';
    print(pathway)
#     pathway = 'OTHER';
    genes = list(Pathway_df_withoutIsRegulated.loc[pathway].dropna());
    pre_res = PreRank(genes, pathway,gene_sets=pathway_filename);
    if(pre_res is None):
        continue; 
    Pathway_df.at[pathway, 'IsRegulated'] = _is_regulated_pathway_(pre_res, pathway);
    print("{} is regulated:{}".format(pathway,_is_regulated_pathway_(pre_res, pathway)))
    PlotEnrichment(pre_res, pathway, outdir=pathway)
    if(pathway in pre_res.res2d.index):
        PlotGSEA(pre_res, pathway,pathway)
        gsea_result_df=pre_res.res2d;
        New_df=New_df.append(gsea_result_df)
    break;
# Pathway_df.to_csv("Pathway_Regulation_status_112821.csv")
# New_df.to_csv("PropionateShunt_Enrichment.csv")

PROPIONATE_SHUNT
Length of genes:5


2022-05-03 16:57:47,186 Parsing data files for GSEA.............................
2022-05-03 16:57:47,383 0001 gene_sets have been filtered out when max_size=500 and min_size=2
2022-05-03 16:57:47,386 0085 gene_sets used for further statistical testing.....
2022-05-03 16:57:47,388 Start to run GSEA...Might take a while..................
2022-05-03 16:57:52,898 Start to generate gseapy reports, and produce figures...


In [32]:
New_df.loc['KETONE_BODY_METABOLISM']

es                                                       0.626754
nes                                                       1.78324
pval                                                   0.00207469
fdr                                                     0.0654537
geneset_size                                                   14
matched_size                                                   14
genes           hphd-1;hach-1;C05C10.3;T02G5.7;sur-5;ech-7;B02...
ledge_genes     hphd-1;hach-1;C05C10.3;T02G5.7;sur-5;ech-7;B02...
Name: KETONE_BODY_METABOLISM, dtype: object

In [30]:
New_df=New_df[New_df.es>0]

In [ ]:
New_df=New_df[New_df.fdr<=0.05]

In [ ]:
# New_df.to_csv("Ketone_body_metabolism.csv")

In [ ]:
New_df.to_csv("KetoneBodyMetabolism.csv")

In [ ]:
Pathway_df.loc['PROPIONATE_SHUNT']

In [ ]:
Pathway_df=pd.read_csv("Pathway_Regulation_status.csv")

In [ ]:
New_df=pd.read_csv("Final_pathway_gsea_033122.csv")

In [ ]:
FDR=New_df[New_df.fdr<=0.05]

In [ ]:
FDR.set_index(['Unnamed: 0'],inplace=True)

In [ ]:
FDR

In [ ]:
FDR.drop(index=['CHITIN_BREAKDOWN','UGT_ENZYME'],inplace=True)

In [ ]:
FDR.to_csv("OverallPathwayEnrichment_032822.csv")

In [ ]:
FDR.shape